# Hand Detection and Cropping Pipeline

## Overview
This notebook demonstrates an automated pipeline for detecting and cropping hand regions from images using **MediaPipe**'s hand detection model.

### Key Features:
- Robust hand detection using MediaPipe
- Automatic cropping with intelligent padding
- Visual output for quality verification
- Batch processing capability

### Use Cases:
- Preparing datasets for gesture recognition
- Sign language processing
- Hand pose estimation preprocessing

## 1. Import Required Libraries

We'll use the following libraries:
- **OpenCV (cv2)**: Image processing and I/O
- **MediaPipe**: Google's hand detection solution
- **NumPy**: Numerical operations
- **Matplotlib**: Visualization

In [ ]:
# Import necessary libraries
import os
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt



## 2. Initialize MediaPipe Hand Detection

### Configuration Parameters:
- **static_image_mode=True**: Optimized for processing static images (not video streams)
- **max_num_hands=1**: Detect only one hand per image
- **min_detection_confidence=0.5**: Minimum confidence threshold (50%)

### How it works:
MediaPipe detects 21 hand landmarks including:
- Wrist (1 point)
- Fingers: Thumb, Index, Middle, Ring, Pinky (4 points each)

In [ ]:
# Initialize MediaPipe Hands solution
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Configure hand detection parameters
hands = mp_hands.Hands(
    static_image_mode=True,      # Process each image independently
    max_num_hands=1,              # Detect only one hand
    min_detection_confidence=0.5  # 50% confidence threshold
)



## 3. Define Hand Cropping Function

### Algorithm Steps:
1. **Convert color space**: BGR → RGB (MediaPipe requirement)
2. **Detect hand landmarks**: Process image through MediaPipe
3. **Calculate bounding box**: Find min/max coordinates from all 21 landmarks
4. **Add padding**: 20 pixels on all sides for better context
5. **Crop image**: Extract the hand region

### Returns:
- Cropped image if hand detected
- `None` if no hand found

In [ ]:
def crop_hand(image):
    """
    Detects the hand and returns the cropped region.
    Returns: cropped_image OR None if no hand detected.
    """

    h, w, _ = image.shape
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if not results.multi_hand_landmarks:
        return None

    # Get bounding box from landmarks
    x_coords = []
    y_coords = []
    for lm in results.multi_hand_landmarks[0].landmark:
        x_coords.append(int(lm.x * w))
        y_coords.append(int(lm.y * h))

    x_min, x_max = max(min(x_coords) - 20, 0), min(max(x_coords) + 20, w)
    y_min, y_max = max(min(y_coords) - 20, 0), min(max(y_coords) + 20, h)

    cropped = image[y_min:y_max, x_min:x_max]
    return cropped


## 4. Test the Pipeline

Let's test our hand detection and cropping pipeline on a sample image.

In [ ]:
test_path = r"D:\Natiq project proposal\preprocessing\clean_dataset\0032\02_02_0032_(15_11_16_17_42_57)_c_0001.jpg"  # change this
img = cv2.imread(test_path)
cropped = crop_hand(img)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title("Original")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1,2,2)
plt.title("Cropped Hand")
if cropped is not None:
    plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
plt.axis('off')


## 5. Batch Processing Example

Process multiple images from a directory and save the cropped results.

In [ ]:
import os
import cv2

input_dataset = r"D:\Natiq project proposal\preprocessing\dataset"
output_dataset = r"D:\Work\Depi\Technical\Natiq\project\clean_dataset"
resize_shape = (224, 224)

os.makedirs(output_dataset, exist_ok=True)

def crop_and_save(img_path, save_folder):
    img = cv2.imread(img_path)
    if img is None:
        return False
    cropped = crop_hand(img)
    if cropped is None:
        return False
    resized = cv2.resize(cropped, resize_shape)
    os.makedirs(save_folder, exist_ok=True)
    save_path = os.path.join(save_folder, os.path.basename(img_path))
    cv2.imwrite(save_path, resized)
    return True

images_saved = 0

for root, dirs, files in os.walk(input_dataset):
    # create a corresponding folder in clean_dataset
    relative_path = os.path.relpath(root, input_dataset)
    save_folder = os.path.join(output_dataset, relative_path)

    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(root, f)
            if crop_and_save(img_path, save_folder):
                images_saved += 1




## 5. Cleanup

Release MediaPipe resources.

In [ ]:
# Close MediaPipe Hands
hands.close()
print("✓ MediaPipe resources released")

## Summary

This preprocessing pipeline successfully:
- Detects hands in images using MediaPipe
- Crops hand regions with appropriate padding
- Processes images in batch mode

